# Demo program using mingo

In [ ]:
from mingo import Database, Hit_distribution
from pathlib import Path

### Create a new database or load an existing one

The basic element of the package is the `Database` object, which allows you to load, create, delete, fill, read and modify a database with simulation results.

The constructor of `Database` --the code that is executed when you create a new object of this class-- starts by looking for a MariaDB database with the given name, `demo` in this case. If the database exists, it will get loaded and, if it does not, the program will ask for permission to create it.

In [ ]:
db = Database(database="demo", use_cnf=True, cnf_path="~/.my.cnf")

There are two ways to initialize a `Database` object: by using a default configuration file or by passing the username, password, host... manually. Whether to use one method or the other is specified via the `use_cnf` param which, by default, is set to True.

### Fill the database from data files

Once you have created a `Database` object, you can use the `fill` and `batch_fill` methods to fill the database using data files. The expected format of the data files is specified in `docs/header-configuration.txt`.

The `fill` method takes the path of a data file as input and tries to fill the database with its contents. A message is printed to indicate that the operation is happening.

The `batch_fill` method allows you to pass multiple source files at once, either as a list of paths to individual data files or as the path to a directory with many data files. In this example, we use `batch_fill` to fill the `demo` database with all the files inside the `doc/demo/sources` directory.

In [ ]:
source_path = Path("sources")
db.batch_fill(source_path)

### Find the ID of plane and detector configurations

Now that the database is ready, there are two ways to read information from it: via methods of the `Database` object or via sql queries using sqlalchemy.

The `Database` object's methods can be used to read certain information with ease, such as the ID of a given plane configuration --more about this in a second. On the other hand, sqlalchemy's queries give you freedom to access all the data in the database at the expense of having to write the queries by yourself. In this demo, I will focus on the first approach, in particular, on the `get_plane_id` and `get_config_id` methods.

Each MiniTrasgo consists on a set of four active planes which may, or may not be preceeded by an absorption plane. Each possible combination of an active and an absorption plane is a 'configuration' (for example: an active plane of 999 x 999 x 22 mm with a lead absorption plane of 10.4 mm placed 22 mm below the first active plane) and, in the database, each configuration is identified by an unique ID.

The `get_plane_id` method allows you to find the ID of a given plane configuration.

In [ ]:
plane_id = db.get_plane_id(999, 999, 22, 22, "Pb", 10.4)

Likewise, each possible combination of plane configurations is a 'detector configuration', for example:
 - First plane (z = 0 mm): No absorption plane.
 - Second plane (z = 100 mm): Lead absorption plane of 10.4 mm at z = 22 mm.
 - Third plane (z = 200 mm): No absorption plane.
 - Fourth plane (z = 400 mm): Lead absorption plane of 16.2 mm at z = 222 mm.

You can find the ID of each of these plane configurations with the `get_plane_id` method. For the `demo` database, they are (1, 2, 1, 3).

You can now use the `get_config_id` method to find the ID of the detector configuration matching the aforementioned settings.

In [ ]:
detector_id = db.get_config_id((1, 2, 1, 3), (0, 100, 200, 400))

### Analize the distribution of hits per event as a function of initial energy

Mingo includes some classes that allow to analyze certain properties of the simulation results with ease.

`Hit_distribution` can be used to analyze the relation between the number of hits per event and the energy of the primary cosmic ray. You initialize it with the `Database` object.

In [ ]:
hits = Hit_distribution(db)

The `__call__` method --the function that is executed when you run `hits()`-- finds the distribution of hits per event and calculates some of its statistical properties.

The results are saved as properties of the `hits` object and can be accesed by calling `hits.dist_data` and `hits.stats_data` respectively.

In [ ]:
hits(detector_id, "NULL - 10.4 - NULL - 16.2")

Once calculated, the `plot_distribution` and `plot_stats` methods allow you to plot the results with an adequate format.

In [ ]:
hit_dist = hits.plot_distribution()
hit_stats = hits.plot_stats()

### Cleanup

You can now eliminate the `demo` database from your MariaDB server by calling the `drop` method of your database object. 

In [ ]:
db.drop()